# Requirements

In [1]:
# Required libraries.
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import spacy
import string
import nltk
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings("ignore")
from spacy import tokenizer
from spacy.lang.en import English
nlp = English()
tkz = tokenizer.Tokenizer(nlp.vocab)
from tqdm import tqdm
from nltk.tokenize import sent_tokenize
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import re
from bs4 import BeautifulSoup
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import unicodedata
import pickle

# The Model(s)

In [ ]:
embedding = pd.read_pickle('/Users/simonefacchiano/Desktop/Data Science/SL/Project/embedding_100k.pkl')

embedding = embedding[['post', 'age', 'gender', 'new_length', 'embedding']]

### Fix the labels for the model

In [7]:
# Notice: there is a little imbalancement here, due to the fact that we are working with the *first* 100k rows of the original dataset. Keep this in mind

embedding['age_class'] = pd.cut(
        embedding["age"],
        bins=[12, 18, 28, 50],
        labels=[0, 1, 2]
    ).astype("int")

In [10]:
embedding['gender'] = embedding['gender'].map({'male': 1, 'female': 0})

In [13]:
# Select and re-order the columns for a better visualization:
embedding = embedding[['post', 'embedding', 'new_length', 'gender', 'age_class']]

embedding.head()

,post,embedding,new_length,gender,age_class
0,ooh shiny new commenting,"[-0.18770814, -0.13998552, 0.019073976, 0.0225...",4,0,0
1,today parade suked wasnt bad band year battle ...,"[-0.110725485, 0.043310776, 0.020091565, -0.06...",23,1,0
2,know anymore concerned everyday want bold face...,"[-0.08226248, -0.03889151, 0.029810807, 0.0900...",38,0,1
3,roof sunset posted paul,"[-0.046943568, 0.13227944, 0.02199171, 0.00461...",4,1,1
4,god love nanny absolutely greatest woman earth...,"[-0.04256986, -0.072457716, 0.005980614, 0.014...",279,0,1


## Predicting the Gender of the author

In [14]:
# Let's first identify the feature matrix and the labels

X = embedding['embedding'].to_list()
y = embedding['gender'].to_list()

In [17]:
# Create the train & validation sets
# In this way, every operation of fine tuning will depend on the data observed in the validation, while the final assessment for the quality of the model will be made on the test set

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, stratify = y) # stratification on the y to avoid class imbalancement

print(f'The train set has {len(X_train)} rows, while the validation set contains {len(X_val)} rows: 80% vs 20%.')

The train set has 80000 rows, while the validation set contains 20000 rows: 80% vs 20%.


In [18]:
# We start with a very simple Logistic Regression to see how it works

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [21]:
from sklearn import metrics

predicted = lr.predict(X_val)

print(f'Logistic Regression Accuracy : {metrics.accuracy_score(y_val, predicted)}')
print(f'Logistic Regression Precision: {metrics.precision_score(y_val, predicted)}')
print(f'Logistic Regression Recall   : {metrics.recall_score(y_val, predicted)}')

####

Logistic Regression Accuracy : 0.6431
Logistic Regression Precision: 0.6484045164457536
Logistic Regression Recall   : 0.6499360299183151
